### Export annotations from Labelbox to GCP storage

In [ ]:
import os, sys
sys.path.insert(0, os.path.abspath('..'))

%load_ext autoreload
%autoreload 2

In [ ]:
from google.cloud import storage
from labelbox import Client as LabelboxClient

from utils import gcp_utils
from utils.data_management import get_annotation_path
from utils.labelbox_utils import get_annotation_objects_from_data_row_export, get_geojson_fc_from_annotation_objects

In [ ]:
# Load environment variables from .env.
# Alternatively, manually set environment variables.

from dotenv import load_dotenv
load_dotenv()

In [ ]:

from project_config import LABELBOX_PROJECT_ID, GCP_PROJECT_NAME
LABELBOX_API_KEY = os.getenv('LABELBOX_API_KEY')

gcp_client = storage.Client(project=GCP_PROJECT_NAME)
labelbox_client = LabelboxClient(api_key=LABELBOX_API_KEY)
project = labelbox_client.get_project(LABELBOX_PROJECT_ID)


In [ ]:
# CAREFUL
# Use with caution

# The following deletes all files in our GCP bucket with a _annotations.geojson file ending

"""
from project_config import BUCKET_NAME

bucket = gcp_client.get_bucket(BUCKET_NAME)
for blob in bucket.list_blobs():
    if blob.name.endswith("_annotations.geojson"):
        print(f"Deleting {blob} .. ")
        blob.delete()
"""

In [ ]:
export_params = {"project_details": True}
export = project.export_v2(params=export_params)

export.wait_till_done()
if export.errors:
  print("Errors:")
  print(export.errors)
else:
  export_full_project = export.result
  print(f"Finished without errors. Total export contains {len(export_full_project)} data rows.")
  print(len(export_full_project))

In [ ]:
from project_config import LABELBOX_PROJECT_ID

for data_row_export in export_full_project:
    global_key_data_row = data_row_export['data_row']['global_key']

    workflow_status = data_row_export['projects'][LABELBOX_PROJECT_ID]['project_details']['workflow_status']
    # We filter depending on workflow_status
    if workflow_status in ["IN_REWORK", "TO_LABEL", "IN_REVIEW"]:
        continue
    assert workflow_status == "DONE"
    
    print(f"Processing annotations for {global_key_data_row}")
    annotated_objects = get_annotation_objects_from_data_row_export(data_row_export)
    
    geojson_feature_collection = get_geojson_fc_from_annotation_objects(annotated_objects)
    annotation_destination_path = get_annotation_path(global_key_data_row)
    gcp_utils.upload_json(gcp_client, geojson_feature_collection, annotation_destination_path)
    